In [1]:
import os
print(os.getcwd())

import warnings
warnings.filterwarnings(action='ignore')

c:\jupyter_home\통합프로젝트\library\code


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('../data/NL_BO_SPECIES_MASTER_NEW_202112.csv')
display(df.head())
df.info()

,master_seq,isbn13,vol,title,author,publisher,pub_date,add_code,price,img_url,description,kdc_class_no,title_replace,author_replace,pub_date_2,is_coll_aladin,is_coll_naver,isbn_origin
0,6352228,9791156759270,NaN,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,이용신 (지은이),푸른숲,NaN,NaN,16000.0,https://image.aladin.co.kr/product/28415/8/cov...,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...,NaN,NaN,NaN,2021-12-03,NaN,NaN,NaN
1,6352229,9791168120877,NaN,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,차도하 (지은이),위즈덤하우스,NaN,NaN,15800.0,https://image.aladin.co.kr/product/28414/66/co...,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...,NaN,NaN,NaN,2021-12-06,NaN,NaN,NaN
2,6352230,9791168120839,NaN,본격 한중일 세계사 12 - 임오군란과 통킹 위기,굽시니스트 (지은이),위즈덤하우스,NaN,NaN,14800.0,https://image.aladin.co.kr/product/28414/47/co...,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...,NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN
3,6352231,9791168120846,NaN,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전,"올리비아 레메스 (지은이), 김잔디 (옮긴이)",위즈덤하우스,NaN,NaN,14000.0,https://image.aladin.co.kr/product/28414/30/co...,"누구에게나 기분 구급상자가 필요하다. 하나씩 하나씩 차근차근, 좋은 기분을 쌓고 건...",NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN
4,6352232,9791168120747,NaN,"오늘도 리추얼 : 음악, 나에게 선물하는 시간",정혜윤 (지은이),위즈덤하우스,NaN,NaN,15000.0,https://image.aladin.co.kr/product/28413/80/co...,나다운 일상을 만드는 사람들의 이야기를 담은 『오늘도 리추얼』 시리즈가 위즈덤하우스...,NaN,NaN,NaN,2021-12-01,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157118 entries, 0 to 157117
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   master_seq      157118 non-null  int64  
 1   isbn13          157118 non-null  int64  
 2   vol             39708 non-null   object 
 3   title           157118 non-null  object 
 4   author          157039 non-null  object 
 5   publisher       151577 non-null  object 
 6   pub_date        0 non-null       float64
 7   add_code        129589 non-null  object 
 8   price           111394 non-null  float64
 9   img_url         98337 non-null   object 
 10  description     84205 non-null   object 
 11  kdc_class_no    143057 non-null  float64
 12  title_replace   148739 non-null  object 
 13  author_replace  149217 non-null  object 
 14  pub_date_2      103518 non-null  object 
 15  is_coll_aladin  154554 non-null  object 
 16  is_coll_naver   154554 non-null  object 
 17  isbn_origi

In [4]:
df2 = df[['title','kdc_class_no','description']]

In [5]:
# 데이터 전처리
def preprocessing_data(df):
    df['title_description'] = df['title'] + " " + df['description']
    df.drop(['title', 'description'], axis=1, inplace=True)
    return df

In [7]:
df2 = preprocessing_data(df2)
display(df2.head())

,kdc_class_no,title_description
0,NaN,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이 2...
1,NaN,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이 “그러니 나는 ...
2,NaN,본격 한중일 세계사 12 - 임오군란과 통킹 위기 한중일 관계의 결정적 분기점인 임...
3,NaN,즉시 기분을 바꿔드립니다 - 신기하게 마음이 편해지는 응급 처방전 누구에게나 기분 ...
4,NaN,"오늘도 리추얼 : 음악, 나에게 선물하는 시간 나다운 일상을 만드는 사람들의 이야기..."


In [8]:
df2 = df2.dropna()
display(df2.head())

,kdc_class_no,title_description
55,811.7,그 너머의 시 - 오하룡 시집 오하룡 시집 『그 너머의 시』는 『시집 밖의 시』를 ...
75,744.0,10일 만에 끝내는 영단어 1000 (상급) 잘 암기되는 단어와 암기가 안 되는 단...
77,234.8,말씀대로 말하라 - 주의 말씀대로 말할 때 하나님의 뜻은 이루어진다 하나님의 말씀대...
322,811.7,반달은 반쪽인가 문예바다 서정시선집 5권. 시인은 “시의 본령은 아무리 시대가 어려...
401,688.0,로건 로건의 마지막 여정을 서부극의 정서로 조율해낸 제임스맨 골드의 탁월한 연출


In [9]:
df2['kdc_class_no'] = df2['kdc_class_no'].astype(int)

In [10]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79238 entries, 55 to 156920
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   kdc_class_no       79238 non-null  int32 
 1   title_description  79238 non-null  object
dtypes: int32(1), object(1)
memory usage: 1.5+ MB


In [11]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
from konlpy.tag import Okt

In [12]:
okt = Okt()

In [13]:
title_description = df2['title_description']

In [14]:
df2['okt_nouns'] = df2['title_description'].apply(okt.nouns)

In [15]:
df2.to_csv('../data/total_book_description_okt.csv')